In [ ]:
import pandas as pd
import geopandas as gpd
import overturemaps 
from shapely import wkb
from lonboard import Map, PolygonLayer
from lonboard.colormap import apply_categorical_cmap

In [ ]:
# specify bounding box
bbox =  -78.6429, 39.463, -73.7806, 41.6242

In [ ]:
# read in Overture Maps land_cover data type
table = overturemaps.record_batch_reader("land_cover", bbox).read_all()
table = table.combine_chunks()

In [ ]:
# convert to dataframe
df = table.to_pandas()

In [2]:
# check shape of dataframe
df.shape

In [ ]:
# filter for higher resolution land_cover features
df_h = df[df.cartography.apply(lambda x: x['min_zoom'] == 8)]

In [ ]:
# check shape of filtered dataframe
df_h.shape

In [ ]:
# create color map for land_cover subtypes, loosely based on natural-color palette: https://www.shadedrelief.com/shelton/c.html
color_map = {
    "urban": [167, 162, 186],
    "forest": [134, 178, 137],
    "barren": [245, 237, 213],
    "shrub": [239, 218, 182],
    "grass": [254, 239, 173],
    "crop": [222, 223, 154],
    "wetland": [158, 207, 195], 
    "mangrove": [83, 171, 128],
    "moss": [250, 230, 160],
    "snow": [255, 255, 255],  
}

In [ ]:
# apply color map to land_cover subtypes
colors = apply_categorical_cmap(df_h.subtype, color_map)

In [ ]:
# dataframe to geodataframe, set crs
gdf = gpd.GeoDataFrame(
    df_h, 
    geometry=df_h['geometry'].apply(wkb.loads), 
    crs="EPSG:4326"
)

In [ ]:
# create map layer 
layer = PolygonLayer.from_geopandas(
    gdf= gdf[['id','subtype', 'cartography', 'geometry']].reset_index(drop=True),
    get_fill_color=colors,
    get_line_color=colors,
)

In [ ]:
#render map
view_state = {
    "longitude": -76.2,
    "latitude": 39.6,
    "zoom": 8,
    "pitch": 65,
    "bearing": 5,
}
m = Map(layer, view_state=view_state)
m